In [29]:
import pandas as pd
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [30]:
import nltk
import re
from nltk.corpus import stopwords

df = pd.read_csv('/content/train.csv', encoding='ISO-8859-1')
df=df.dropna()
print(df.head(5))
x=df.drop(columns='label')
y=df['label']
voc_size=5000
messages=x.copy()
messages.reset_index(inplace=True)
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

   id                                              title              author  \
0   0  House Dem Aide: We Didnât Even See Comeyâs...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didnât Even See Comeyâs...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [32]:
for i in range(0, len(messages)):
    print(i)
    if isinstance(messages['title'][i], str):  # Check if the entry is a string
        review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    else:
        # Handle NaN or non-string values
        corpus.append('')  # You can choose to skip these entries or handle them differently


Streaming output truncated to the last 5000 lines.
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
12945
12946
12947
12948
12949
12950
12951
12952
12953
12954
12955
12956
12957
12958
12959
12960
12961
12962
12963
12964
12965
12966
12967
12968
12969
12970
12971
12972
12973
12974
12975
12976
12977
12978
12979
12980
12981
12982
12983
12984
12985
12986
12987
12988
12989
12990
12991
12992
12993
12994
12995
12996
12997
12998
12999
13000
13001
13002
13003
13004
13005
13006
13007
13008
13009
13010
13011
13012
13013
13014
13015
13016
13017
13018
13019
13020
13021
13022
13023
13024
13025
13026
13027
13028
13029
13030
13031
13032
13033
13034
13035
13036
13037
13038
13039
13040
13041
13042
13043
13044
13045
13046
1

In [33]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]  # Convert generator expression to list comprehension
print(onehot_rep)

sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[587, 1587, 843, 4955, 2520, 2239, 2888, 3164, 348, 1499], [655, 2204, 503, 3437, 1816, 1972, 2088], [1035, 94, 3537, 4726], [2664, 4326, 3508, 1884, 4692, 2551], [1579, 1816, 2717, 3653, 620, 2807, 1816, 459, 2359, 3282], [4020, 3375, 4281, 2023, 3129, 3762, 3332, 233, 4593, 3114, 2302, 2383, 4009, 2364, 2088], [402, 4122, 3801, 2792, 2995, 4538, 3711, 4391, 3942, 2021, 3324], [2458, 2955, 775, 1220, 3497, 3450, 3762, 2692, 3942, 2021, 3324], [1402, 4863, 34, 3883, 2229, 1361, 2128, 4216, 3762, 42], [4990, 3539, 3350, 550, 828, 2071, 265, 1127], [1337, 1024, 4012, 850, 4847, 1214, 3945, 2581, 963, 2062, 265], [1884, 4231, 2520, 1361, 3762, 3497], [2850, 830, 3423, 2377, 3375, 4977, 658, 165, 3313], [3148, 1496, 3445, 149, 1006, 3572, 901, 3942, 2021, 3324], [4763, 3541, 3141, 3479, 2394, 3942, 2021, 3324], [4218, 349, 4879, 4067, 3643, 329, 4551, 1626, 1358, 1946], [1232, 3029, 2204], [4020, 4541, 2545, 3560, 3762, 2674, 142, 2088], [4030, 258, 503, 3605, 131, 3354, 1348, 392, 2807],

In [34]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))

In [35]:
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_2 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [36]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [37]:
print(X_final.shape)
print(y_final.shape)



(17889, 20)
(17889,)


In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
188/188 [==============================] - 19s 102ms/step - loss: 0.0084 - accuracy: 0.9978 - val_loss: 0.5886 - val_accuracy: 0.9002
Epoch 2/10
188/188 [==============================] - 14s 75ms/step - loss: 0.0055 - accuracy: 0.9991 - val_loss: 0.6708 - val_accuracy: 0.9004
Epoch 3/10
188/188 [==============================] - 16s 86ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 0.6188 - val_accuracy: 0.9045
Epoch 4/10
188/188 [==============================] - 15s 81ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 0.6791 - val_accuracy: 0.9031
Epoch 5/10
188/188 [==============================] - 14s 77ms/step - loss: 0.0063 - accuracy: 0.9981 - val_loss: 0.6542 - val_accuracy: 0.9028
Epoch 6/10
188/188 [==============================] - 14s 76ms/step - loss: 0.0057 - accuracy: 0.9981 - val_loss: 0.6940 - val_accuracy: 0.8989
Epoch 7/10
188/188 [==============================] - 15s 82ms/step - loss: 0.0079 - accuracy: 0.9977 - val_loss: 0.5729 - val_accuracy

In [48]:
y_pred1=model1.predict(X_test)
from sklearn.metrics import accuracy_score
y_pred_classes = np.argmax(y_pred1, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy:", accuracy)

185/185 [==============================] - 3s 14ms/step
Accuracy: 0.5633468834688347
